In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

# 학습 데이터와 테스트 데이터 읽기
train_file_path = '/content/train.csv'  # 학습 데이터 파일 경로
test_file_path = '/content/test.csv'      # 테스트 데이터 파일 경로
train_data = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)

# 결측값 처리
train_data.fillna('WT', inplace=True)
test_data.fillna('WT', inplace=True)

# 특징과 타겟 분리
X_train = train_data.drop(columns=['ID', 'SUBCLASS'])  # ID와 SUBCLASS 열 제거
y_train = train_data['SUBCLASS']

# 특징 인코딩: 'WT'는 0, 그 외는 첫 글자 + 컬럼명 조합으로 변환
def encode_feature(col):
    return col.apply(lambda x: '' if x == 'WT' else f"{x[0]}_{col.name}")

# 모든 특성에 대해 인코딩 적용
X_train_encoded = X_train.apply(encode_feature)
X_test_encoded = test_data.drop(columns=['ID']).apply(encode_feature)  # 테스트 데이터 인코딩

# 레이블 인코딩
le = LabelEncoder()
for c in X_train_encoded.columns:
    X_train_encoded[c] = le.fit_transform(X_train_encoded[c])

# 테스트 데이터에도 동일한 인코딩 적용
for c in X_test_encoded.columns:
    if c in le.classes_:
        X_test_encoded[c] = le.transform(X_test_encoded[c])
    else:
        X_test_encoded[c] = le.fit_transform(X_test_encoded[c])

# 레이블 인코딩
le_y = LabelEncoder()
y_encoded = le_y.fit_transform(y_train)

# 훈련 데이터와 검증 데이터 분리
X_train_final, X_val, y_train_final, y_val = train_test_split(X_train_encoded, y_encoded, test_size=0.2, random_state=42)

# NumPy 배열로 변환
X_train_final = X_train_final.values
X_val = X_val.values
X_test_encoded = X_test_encoded.values

# 모델 구성
model = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X_train_final.shape[1],)),
    layers.Dropout(0.4),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(le_y.classes_), activation='softmax')  # 클래스 수에 맞추어 출력 노드 수 설정
])

# 모델 컴파일
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 모델 학습
model.fit(X_train_final, y_train_final, validation_data=(X_val, y_val),
          epochs=200, batch_size=32,
          callbacks=[early_stopping])

# 검증 데이터 평가
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

# 테스트 데이터 예측
test_predictions = model.predict(X_test_encoded)

# 예측 결과 변환
predicted_labels = le_y.inverse_transform(test_predictions.argmax(axis=1))

# 결과를 DataFrame으로 저장
results = pd.DataFrame({
    'ID': test_data['ID'],
    'SUBCLASS': predicted_labels
})

# 결과를 CSV 파일로 저장
results.to_csv('predictions_combined_data.csv', index=False)

print("Predictions saved to 'predictions_combined_data.csv'.")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.1269 - loss: 3.1852 - val_accuracy: 0.2627 - val_loss: 2.6081
Epoch 2/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - accuracy: 0.3300 - loss: 2.3076 - val_accuracy: 0.2877 - val_loss: 2.3395
Epoch 3/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.5059 - loss: 1.5872 - val_accuracy: 0.3022 - val_loss: 2.2999
Epoch 4/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.6572 - loss: 1.0505 - val_accuracy: 0.3038 - val_loss: 2.4230
Epoch 5/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.7403 - loss: 0.7671 - val_accuracy: 0.3046 - val_loss: 2.6603
Epoch 6/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.8010 - loss: 0.5951 - val_accuracy: 0.2933 - val_loss: 2.8113
Epoch 7/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - accuracy: 0.8323 - loss: 0.5183 - val_accuracy: 0.2965 - val_loss: 2.9689
Epoch 8/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.8594 - loss: 0.4128 - 

In [10]:
results.SUBCLASS.value_counts()

,count
SUBCLASS,
STES,745
BRCA,503
KIPAN,204
COAD,153
THCA,123
GBMLGG,104
SKCM,93
PRAD,93
LGG,90
